In [ ]:
%matplotlib inline

import pysia as sia
import numpy as np
import matplotlib.pyplot as plt

system = sia.LinearGaussian(
    A=np.array([[1, -dt], [0, 1]]),
    B=np.array([[dt], [0]]),
    C=np.array([[1, 0], [0, 1]]),
    H=np.array([[1, 0]]),
    Q=np.array([[q00, q01], [q01, q11]]),
    R=np.array([[sn**2]]))

# Let's the write the particle filter for a Gaussian with what we have
estimate = sia.Gaussian(mean=np.array([0, 0]), covariance=np.diag([1E-4, 1E-12]))

n_steps = 60
n_particles = 1000

xp = np.uniform(-1, 1, n_particles)
wp = np.ones((n_particles,)) / n_particles

# Generic particle filter
for k in range(1, n_steps):
    u = 0
    y = 0

    for i in range(n_particles):
        # Sample from the dynamics
        xp[i,:] = system.dynamics(xp[i,:], u).sample()
    
        # Compute the likelihoods
        lp[i,:] = system.measurement(xp[i,:], u).logProb(y)
        
    # Reweight
    wp = wp * np.exp(lp)
    wp = wp / wp.sum()
    
    # Resample...
    # If Neff < N
    #    systematic...